# Module: braggdiskdetection

This module contains functions finding the positions of the Bragg disks in a 4DSTEM scan.  Generally this will involve two steps: getting a vacuum probe, then finding the Bragg disks using the vacuum probe as a template. 

## Submodule: diskdetection

The notebook demos functions related to finding the Bragg disks.  Using a vacuum probe as a template - i.e. a convolution kernel - a cross correlation (or phase or hybrid correlation) is taken between each DP and the template, and the positions and intensities of all local corraltion maxima are used to identify the Bragg disks.  Erroneous peaks are filtered out with several types of threshold.  Detected Bragg disks are generally stored in PointLists (when run on only selected DPs) or PointListArrays (when run on a full DataCube).

This notebook demos:
* Disk detection on single or selected diffraction patterns
* Disk detection on all diffraction patterns
* Additional filtering of detected Bragg disks

### Import packages, load data

In [ ]:
import py4DSTEM

import numpy as np
import matplotlib.pyplot as plt

from py4DSTEM.process.braggdiskdetection import find_Bragg_disks_single_DP
from py4DSTEM.process.braggdiskdetection import find_Bragg_disks_selected
from py4DSTEM.process.braggdiskdetection import find_Bragg_disks
from py4DSTEM.process.braggdiskdetection import threshold_Braggpeaks


In [ ]:
import py4DSTEM.disk_detection_parallel

from py4DSTEM.disk_detection_parallel import find_Bragg_disks_ipp

In [ ]:
# Load data
#fp = "/home/ben/Data/20180905_FePO4_unlithiated/raw/Stack1_57x47+30nmss_spot 8_0p05s_CL=600_alpha=0p48_300kV_bin4.dm4"
#fp = "/Users/Ben/Work/NCEM/Projects/py4DSTEM/sample_data/20180905_FePO4_unlithiated/Stack2_60x60+30nmss_spot 8_0p05s_CL=600_alpha=0p48_300kV_bin4.dm3"

#fp = "/global/project/projectdirs/ncemhub/py4DSTEM-test-data/Cuscan7.dm4"
fp = "/global/u2/m/mhenders/ncem/Stack2_60x60+30nmss_spot 8_0p05s_CL=600_alpha=0p48_300kV_bin4.h5"
dc = py4DSTEM.file.io.read(fp)
dc.set_scan_shape(47,57)
#dc.data4D = np.roll(dc.data4D,-2,1) # Correct for acquisition wrap-around error

# Load the template
#fp_probetemplate = "/home/ben/Data/20180905_FePO4_unlithiated/processing/vacuum_probe_kernel.h5"
#fp = "/Users/Ben/Work/NCEM/Projects/py4DSTEM/sample_data/20180905_FePO4_unlithiated/processing/vacuum_probe_kernel.h5"
fp_probetemplate = "/global/u2/m/mhenders/ncem/vacuum_probe_kernel.h5"
browser = py4DSTEM.file.io.FileBrowser(fp_probetemplate, rawdatacube=dc)
browser.show_dataobjects()
probe_kernel = browser.get_dataobject(0)

#### Single DP

In [ ]:
# Select a DP

Rx=20
Ry=25
power=0.3

DP = dc.data4D[Rx,Ry,:,:]

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,12))
ax1.matshow(np.average(dc.data4D,axis=(2,3)))
ax1.scatter(Ry,Rx,color='r')
ax2.matshow(DP**power)
ax1.axis('off')
ax2.axis('off')
plt.show()

In [ ]:
# Get peaks

corrPower = 0.9
sigma = 2
edgeBoundary = 20
maxNumPeaks = 70
minPeakSpacing = 30
minRelativeIntensity = 0.005

peaks = find_Bragg_disks_single_DP(DP, probe_kernel.data2D,
                                   corrPower=corrPower,
                                   sigma=sigma,
                                   edgeBoundary=edgeBoundary,
                                   minRelativeIntensity=minRelativeIntensity,
                                   minPeakSpacing=minPeakSpacing,
                                   maxNumPeaks=maxNumPeaks)

In [ ]:
# Show

power=0.3
size_scale_factor = 500       # Set to zero to make all points the same size

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,12))
ax1.matshow(np.average(dc.data4D,axis=(2,3)))
ax1.scatter(Ry,Rx,color='r')
ax2.matshow(DP**power)
ax2.scatter(peaks.data['qy'],peaks.data['qx'],color='r',s=size_scale_factor*peaks.data['intensity']/np.max(peaks.data['intensity']))
ax1.axis('off')
ax2.axis('off')
plt.show()

#### Several DPs

In [ ]:
# Select a few DPs

Rxs=(20,31,18)
Rys=(25,31,10)
power=0.3

fig,((ax11,ax12),(ax21,ax22))=plt.subplots(2,2,figsize=(12,12))
ax11.matshow(np.average(dc.data4D,axis=(2,3)))
ax11.scatter(Rys,Rxs,color=('r','yellow','deepskyblue'))
ax12.matshow(dc.data4D[Rxs[0],Rys[0],:,:]**power)
ax21.matshow(dc.data4D[Rxs[1],Rys[1],:,:]**power)
ax22.matshow(dc.data4D[Rxs[2],Rys[2],:,:]**power)

ax11.axis('off')
ax12.axis('off')
ax21.axis('off')
ax22.axis('off')
plt.show()

In [ ]:
# Get peaks

corrPower = 0.8
sigma = 2
edgeBoundary = 20
maxNumPeaks = 70
minPeakSpacing = 50
minRelativeIntensity = 0.001

peaks = find_Bragg_disks_selected(dc, probe_kernel.data2D, Rxs, Rys,
                                  corrPower=corrPower,
                                  sigma=sigma,
                                  edgeBoundary=edgeBoundary,
                                  minRelativeIntensity=minRelativeIntensity,
                                  minPeakSpacing=minPeakSpacing,
                                  maxNumPeaks=maxNumPeaks)

In [ ]:
# Show

power=0.3
size_scale_factor = 500       # Set to zero to make all points the same size

fig,((ax11,ax12),(ax21,ax22))=plt.subplots(2,2,figsize=(12,12))
ax11.matshow(np.average(dc.data4D,axis=(2,3)))
ax11.scatter(Rys,Rxs,color=('r','g','b'))
ax12.matshow(dc.data4D[Rxs[0],Rys[0],:,:]**power)
ax21.matshow(dc.data4D[Rxs[1],Rys[1],:,:]**power)
ax22.matshow(dc.data4D[Rxs[2],Rys[2],:,:]**power)

if size_scale_factor == 0:
    ax12.scatter(peaks[0].data['qy'],peaks[0].data['qx'],color='r')
    ax21.scatter(peaks[1].data['qy'],peaks[1].data['qx'],color='g')
    ax22.scatter(peaks[2].data['qy'],peaks[2].data['qx'],color='b')
else:
    ax12.scatter(peaks[0].data['qy'],peaks[0].data['qx'],color='r',s=size_scale_factor*peaks[0].data['intensity']/np.max(peaks[0].data['intensity']))
    ax21.scatter(peaks[1].data['qy'],peaks[1].data['qx'],color='g',s=size_scale_factor*peaks[1].data['intensity']/np.max(peaks[1].data['intensity']))
    ax22.scatter(peaks[2].data['qy'],peaks[2].data['qx'],color='b',s=size_scale_factor*peaks[2].data['intensity']/np.max(peaks[2].data['intensity']))


ax11.axis('off')
ax12.axis('off')
ax21.axis('off')
ax22.axis('off')
plt.show()

In [ ]:
job_id = open('slurm_jobid', 'r').read().strip()
cluster_id = "cori_{}".format(job_id)

In [ ]:
import ipyparallel as ipp
c = ipp.Client(cluster_id=cluster_id)
print(c.ids)

#### All DPs

In [ ]:
# Get peaks
from time import time

corrPower = 0.8
sigma = 2
edgeBoundary = 20
maxNumPeaks = 70
minPeakSpacing = 50
minRelativeIntensity = 0.001
verbose = False
subpixel = 'poly'
#subpixel = 'multicorr'
#upsample_factor=16

t0 = time()
peaks = find_Bragg_disks_ipp(
    dc, probe_kernel.data2D,
    corrPower=corrPower,
    sigma=sigma,
    edgeBoundary=edgeBoundary,
    minRelativeIntensity=minRelativeIntensity,
    minPeakSpacing=minPeakSpacing,
    maxNumPeaks=maxNumPeaks,
    verbose=verbose,
    subpixel=subpixel,
    #upsample_factor=upsample_factor,
    cluster_id=cluster_id)
overall_time = time() - t0
print(overall_time)

In [ ]:
# Show

Rxs=(20,31,18)
Rys=(25,31,10)
power=0.3
size_scale_factor = 500       # Set to zero to make all points the same size

fig,((ax11,ax12),(ax21,ax22))=plt.subplots(2,2,figsize=(12,12))
ax11.matshow(np.average(dc.data4D,axis=(2,3)))
ax11.scatter(Rys,Rxs,color=('r','g','b'))
ax12.matshow(dc.data4D[Rxs[0],Rys[0],:,:]**power)
ax21.matshow(dc.data4D[Rxs[1],Rys[1],:,:]**power)
ax22.matshow(dc.data4D[Rxs[2],Rys[2],:,:]**power)

peaks0 = peaks.get_pointlist(Rxs[0],Rys[0])
peaks1 = peaks.get_pointlist(Rxs[1],Rys[1])
peaks2 = peaks.get_pointlist(Rxs[2],Rys[2])
if size_scale_factor == 0:
    ax12.scatter(peaks0.data['qy'],peaks0.data['qx'],color='r')
    ax21.scatter(peaks1.data['qy'],peaks1.data['qx'],color='g')
    ax22.scatter(peaks2.data['qy'],peaks2.data['qx'],color='b')
else:
    ax12.scatter(peaks0.data['qy'],peaks0.data['qx'],color='r',s=size_scale_factor*peaks0.data['intensity']/np.max(peaks0.data['intensity']))
    ax21.scatter(peaks1.data['qy'],peaks1.data['qx'],color='g',s=size_scale_factor*peaks1.data['intensity']/np.max(peaks1.data['intensity']))
    ax22.scatter(peaks2.data['qy'],peaks2.data['qx'],color='b',s=size_scale_factor*peaks2.data['intensity']/np.max(peaks2.data['intensity']))

ax11.axis('off')
ax12.axis('off')
ax21.axis('off')
ax22.axis('off')
plt.show()

#### Apply post-detection thresholding

In [ ]:
# Remove points based on new peak spacing or minimum relative intensity thresholds

maxNumPeaks = 20
minPeakSpacing = 50
minRelativeIntensity = 0.01

peaks_thresh = peaks.copy(name='Braggpeaks')  # Create a copy of the PointListArray to further threshold
peaks_thresh = threshold_Braggpeaks(peaks_thresh,
                                    minRelativeIntensity=minRelativeIntensity,
                                    minPeakSpacing=minPeakSpacing,
                                    maxNumPeaks=maxNumPeaks)

In [ ]:
# Show

Rxs=(20,31,18)
Rys=(25,31,10)
power=0.3
size_scale_factor = 500       # Set to zero to make all points the same size

fig,((ax11,ax12),(ax21,ax22))=plt.subplots(2,2,figsize=(12,12))
ax11.matshow(np.average(dc.data4D,axis=(2,3)))
ax11.scatter(Rys,Rxs,color=('r','g','b'))
ax12.matshow(dc.data4D[Rxs[0],Rys[0],:,:]**power)
ax21.matshow(dc.data4D[Rxs[1],Rys[1],:,:]**power)
ax22.matshow(dc.data4D[Rxs[2],Rys[2],:,:]**power)

peaks0 = peaks_thresh.get_pointlist(Rxs[0],Rys[0])
peaks1 = peaks_thresh.get_pointlist(Rxs[1],Rys[1])
peaks2 = peaks_thresh.get_pointlist(Rxs[2],Rys[2])
if size_scale_factor == 0:
    ax12.scatter(peaks0.data['qy'],peaks0.data['qx'],color='r')
    ax21.scatter(peaks1.data['qy'],peaks1.data['qx'],color='g')
    ax22.scatter(peaks2.data['qy'],peaks2.data['qx'],color='b')
else:
    ax12.scatter(peaks0.data['qy'],peaks0.data['qx'],color='r',s=size_scale_factor*peaks0.data['intensity']/np.max(peaks0.data['intensity']))
    ax21.scatter(peaks1.data['qy'],peaks1.data['qx'],color='g',s=size_scale_factor*peaks1.data['intensity']/np.max(peaks1.data['intensity']))
    ax22.scatter(peaks2.data['qy'],peaks2.data['qx'],color='b',s=size_scale_factor*peaks2.data['intensity']/np.max(peaks2.data['intensity']))

ax11.axis('off')
ax12.axis('off')
ax21.axis('off')
ax22.axis('off')
plt.show()